In [20]:
import os
import pickle
import networkx as nx
from tqdm import tqdm
import torch
import numpy as np
import math

from torch_geometric.datasets import WebKB, WikipediaNetwork, Actor, Planetoid, HeterophilousGraphDataset
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, from_networkx, to_dense_adj
import torch_geometric.transforms as T
from torch_geometric.transforms import LargestConnectedComponents, ToUndirected
from torch_geometric.loader import DataLoader

from sklearn.cluster import SpectralClustering

In [39]:
def generate_spectral_clustering_graph(num_nodes: int, topology: str="complete") -> Data:
    assert num_nodes > 0
    assert topology in ["complete", "path", "cycle", "regular", "tree"], "Error: unknown topology"  # Extend this list for other topologies
    
    # Create a networkx graph with the desired topology
    if topology == "complete":
        raw_graph = create_complete_graph(num_nodes)

    if topology == "path":
        raw_graph = create_path_graph(num_nodes)

    if topology == "cycle":
        raw_graph = create_cycle_graph(num_nodes)

    if topology == "regular":
        raw_graph = create_4_regular_grid_graph(num_nodes, num_nodes)

    if topology == "tree":
        raw_graph = create_binary_tree(num_nodes)

    # Randomly select two nodes to be relevant
    relevant_nodes = np.random.choice(raw_graph.nodes(), 2, replace=False)
    
    # Add features to nodes: 1 for relevant nodes, 0 for others
    for node in raw_graph.nodes():
        raw_graph.nodes[node]['feature'] = 1 if node in relevant_nodes else 0

    # Convert the NetworkX graph to PyTorch Geometric's Data format
    attributed_graph = from_networkx(raw_graph)
    
    # Perform spectral clustering
    adjacency_matrix = nx.to_numpy_array(raw_graph)
    clustering = SpectralClustering(n_clusters=2, affinity='precomputed')
    labels = clustering.fit_predict(adjacency_matrix)
    print(labels)
    
    # Check if the two relevant nodes are in the same cluster
    same_cluster = labels[relevant_nodes[0]] == labels[relevant_nodes[1]]
    
    # Set the graph label to 1 if in the same cluster, otherwise 0
    attributed_graph.y = torch.tensor([1 if same_cluster else 0], dtype=torch.float)
    
    return attributed_graph

In [9]:
# topologies

def create_complete_graph(num_nodes: int) -> nx.graph:
    complete_graph = nx.complete_graph(num_nodes).to_undirected()
    return complete_graph

def create_path_graph(num_nodes: int) -> nx.Graph:
    path_graph = nx.path_graph(num_nodes)
    return path_graph

def create_cycle_graph(num_nodes: int) -> nx.Graph:
    cycle_graph = nx.cycle_graph(num_nodes)
    return cycle_graph

def create_4_regular_grid_graph(rows: int, cols: int) -> nx.Graph:    
    grid_graph = nx.grid_2d_graph(rows, cols, periodic=True)  # Wraps around for 4-regular structure
    grid_graph =  nx.convert_node_labels_to_integers(grid_graph)
    for node in grid_graph.nodes:
        grid_graph.nodes[node].clear()
    return grid_graph

def create_binary_tree(num_nodes: int) -> nx.Graph:
    max_depth = math.ceil(math.log2(num_nodes + 1)) - 1
    tree = nx.balanced_tree(r=2, h=max_depth)    
    return tree